In [5]:
from hydrology import HydrologyApi, Measure
from datetime import datetime
import polars as pl

## Load Dataset

In [6]:
api = HydrologyApi()
level_stations = api.get_stations(Measure.MeasureType.LEVEL, river="River Wear")
rainfall_stations = api.get_stations(
    Measure.MeasureType.RAINFALL, position=(54.774, -1.558), radius=15
)


measures = [Measure(station_id, Measure.MeasureType.LEVEL) for station_id in level_stations['station_id']] + [
    Measure(station_id, Measure.MeasureType.RAINFALL) for station_id in rainfall_stations['station_id']
]

stations = pl.concat(
    [
        level_stations,
        rainfall_stations,
    ],
).unique()

df = api.get_measures(measures, stations, start_date=datetime(2007, 1, 1))

df.head()

timestamp,Durham New Elvet Bridge level-i-900-m,North Dalton rainfall-t-900-mm,Sunderland Bridge level-i-900-m,Chester Le Street level-i-900-m,Knitlsey Mill rainfall-t-900-mm,Witton Park level-i-900-m,Peterlee rainfall-t-900-mm,Evenwood Gate rainfall-t-900-mm,Fulwell rainfall-t-900-mm,Stanhope level-i-900-m,Tunstall rainfall-t-900-mm,Harpington Hill Farm rainfall-t-900-mm,Washington rainfall-t-900-mm,Stanley Hustledown rainfall-t-900-mm,ESH Winning rainfall-t-900-mm
datetime[μs],f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2007-01-01 00:00:00,0.726,0.0,0.851,0.824,0.0,1.009,0.0,0.0,0.0,0.859,0.0,0.0,null,null,null
2007-01-01 00:15:00,0.73,0.0,0.863,0.821,0.0,0.997,0.0,0.0,0.0,0.855,0.0,0.0,null,null,null
2007-01-01 00:30:00,0.74,0.0,0.876,0.823,0.0,1.0,0.0,0.0,0.0,0.845,0.0,0.0,null,null,null
2007-01-01 00:45:00,0.744,0.0,0.886,0.819,0.0,1.001,0.0,0.0,0.0,0.826,0.0,0.0,null,null,null
2007-01-01 01:00:00,0.763,0.0,0.894,0.823,0.0,0.993,0.0,0.0,0.0,0.825,0.0,0.0,null,null,null


## Forecasting

In [ ]:
from neuralforecast.core import NeuralForecast
from neuralforecast.auto import NHITS
from neuralforecast.losses.pytorch import MQLoss

raise NotImplementedError("This code is not yet implemented")

train_df = (
  df
  .reset_index()
  .rename_axis(None, axis=1)
  .rename(columns={
    "Durham New Elvet Bridge (level)": "y",
    "timestamp": "ds",
  })
  .assign(unique_id = "River Wear")
)

In [ ]:
def make_loss():
  return MQLoss(quantiles=[0.5, 0.9, 0.99])

models = [
  NHITS(
    h = 4 * 24, # 1 day
    input_size = 3 * 4 * 24, # 3 days
    hist_exog_list = train_df.columns.drop(["ds", "unique_id", "y"]).to_list(),
    scaler_type = 'robust',
    loss=make_loss(),
    max_steps=1000,
  )
]

nf = NeuralForecast(
  models = models,
  freq = '15min',
)

nf.fit(df=train_df)